This script rips zalando for sweaters and sorts the results based in wool content in the sweaters

In [ ]:
from __future__ import print_function, division
import urllib2
from bs4 import BeautifulSoup
import re
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def getPageSoup(pg):
    baseUrl = 'https://www.zalando.dk/herretoej-sweatere-og-strikjakker/?price_to=500&upper_material=kashmir.mohair.uld&order=price&dir=asc'
    # html = urllib2.urlopen(baseUrl % pg).read()
    html = urllib2.urlopen(baseUrl).read()
    return BeautifulSoup(html, 'lxml')

In [ ]:
soup = getPageSoup(1)

In [ ]:
soup = BeautifulSoup(urllib2.urlopen('https://www.zalando.dk/herretoej-sweatere-og-strikjakker/?upper_material=uld&shirt_collar=sjalskrave&order=price&dir=asc').read(), 'lxml')

In [ ]:
def getPageProducts(soup):
    searchBox = soup.find('ul', class_='catalogArticlesList  threeCol main')
    productList = searchBox.findAll('a', class_='catalogArticlesList_productBox')
    productLinks = ['https://www.zalando.dk' + a.attrs['href'] for a in productList]
    return productLinks

In [ ]:
productLinks = getPageProducts(soup)

In [ ]:
def getProductSoup(url):
    html = urllib2.urlopen(url).read()
    return BeautifulSoup(html, 'lxml')

In [ ]:
def getWollContent(soup):
    productInfo = soup.find('div', attrs={'id': 'productDetails', 'class': 'span6'})
    ul = productInfo.find('ul')
    rx = re.compile(r'.*?(\d+)\s*\% uld.*?')
    for ln in ul.text.splitlines():
        m = rx.match(ln)
        if m:
            return int(m.groups()[0].strip())
    return -1

In [ ]:
aggregateList = list()
for pg in range(1,6):
    pageSoup = getPageSoup(pg)
    productLinks = getPageProducts(pageSoup)
    for pd in productLinks:
        try:
            prodSoup = getProductSoup(pd)
            woolCont = getWollContent(prodSoup)
            aggregateList.append((woolCont, pd))
        except AttributeError as e:
            print('Failed at URL %s ' % pd)
            print(e)

In [ ]:
len(aggregateList)

In [ ]:
%matplotlib inline
mpl.rcParams['figure.figsize'] = (10.0, 8.0)
pct = [el[0] for el in aggregateList]
plt.hist(pct, 100);
plt.xticks(range(0, 101, 5));
plt.xlabel('% wool');
plt.ylabel('# products');

In [ ]:
with open('woolen_sweaters_urls.txt', 'w') as fid:
    map(lambda x: print(x[1], file=fid), sorted([el for el in aggregateList if el[0] > 50], key=lambda x: x[0], reverse=True));
map(lambda x: print(x[1]), sorted([el for el in aggregateList if el[0] > 50], key=lambda x: x[0], reverse=True));

In [ ]:
map(lambda x:print(x[1]), [el for el in aggregateList if el[0] == 50]);

In [ ]:
# import webbrowser
# map(lambda x: webbrowser.open_new_tab(x[1]), [el for el in aggregateList if el[0] > 50])